# Segmenting and Clustering Neighborhoods in Toronto - Explore and Cluster

## We are basically adding to the the notebook from pt 2

In [1]:
import pandas as pd

In [2]:
latlong = pd.read_csv("http://cocl.us/Geospatial_data/") 
latlong.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [3]:
#Rename "Postal Code" with "PostalCode"
latlong.columns = ['PostalCode','Latitude','Longitude']
latlong.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Make sure there are 103 postal codes as we determined from previous assignment

In [4]:
latlong.shape[0]

103

#### We use the code from the Notebook SCN-Toronto-Scrape-Wiki.ipynb to get the postcode, borough and neigborhood (previous assignment)

In [5]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
table = pd.read_html(url, header=0,keep_default_na=False) 
tdf = table[0]
tdf.columns = ['PostalCode','Borough','Neighborhood']
tdf = tdf.query('Borough != "Not assigned"').reset_index(drop=True)
tdf1=tdf.groupby('PostalCode', as_index=False).agg(lambda x: ', '.join(set(x.dropna())))
tdf1.loc[tdf1['Neighborhood'] == 'Not assigned', 'Neighborhood' ] = tdf1['Borough']
tdf1.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Port Union, Highland Creek, Rouge Hill"
2,M1E,Scarborough,"West Hill, Guildwood, Morningside"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


#### We merge the two tables with latlong on the right

In [6]:
tdf2 = pd.merge(left=tdf1, right=latlong, on="PostalCode", how="right")
tdf2.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Port Union, Highland Creek, Rouge Hill",43.784535,-79.160497
2,M1E,Scarborough,"West Hill, Guildwood, Morningside",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Let us now explore Toronto

#### Due to ToS with respect to User Agent related to Nominatim, we directly use lat and long values below
#### Map of Toronto with neighborhoods superimposed

In [7]:
import folium 

# create map of York using latitude and longitude values

map_Toronto = folium.Map(location=[43.761539, -79.411079], zoom_start=11)

# add markers to map
for lat, lng, bor, nei in zip(tdf2['Latitude'], tdf2['Longitude'], tdf2['Borough'], tdf2['Neighborhood']):
    
    label = '{}, {}'.format(nei, bor)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_Toronto)  
    
map_Toronto

#### Define Foursquare Credentials and Version

In [8]:
###Removing CLIENT_ID and CLIENT_SECRET after run

CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

#### Examine Neighborhoods in Scarborough, lat-long 43.773077, -79.257774

In [9]:
Scarborough_data = tdf2[tdf2['Borough'] == 'Scarborough'].reset_index(drop=True)
map_scarborough = folium.Map(location=[43.773077, -79.257774], zoom_start=11)
for lat, lng, bor, nei in zip(Scarborough_data['Latitude'], Scarborough_data['Longitude'],Scarborough_data['Borough'], Scarborough_data['Neighborhood']):
    
    label = '{}, {}'.format(nei, bor)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_scarborough)  
    
map_scarborough

#### We find nearby venues in Scarborough

In [10]:
import requests
LIMIT = 100

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        venue_results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in venue_results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)


scarborough_venues = getNearbyVenues(names=Scarborough_data['Neighborhood'],
                                   latitudes=Scarborough_data['Latitude'],
                                   longitudes=Scarborough_data['Longitude']
                                  )


Rouge, Malvern
Port Union, Highland Creek, Rouge Hill
West Hill, Guildwood, Morningside
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Oakridge, Golden Mile, Clairlea
Cliffside, Scarborough Village West, Cliffcrest
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, Steeles East, Milliken, L'Amoreaux East
L'Amoreaux West, Steeles West
Upper Rouge


## Now we cluster and visulaize neighborhoods

In [11]:
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Agincourt North, Steeles East, Milliken, L'Amoreaux East",3,3,3,3,3,3
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Cedarbrae,8,8,8,8,8,8
"Clarks Corners, Sullivan, Tam O'Shanter",9,9,9,9,9,9
"Cliffside, Scarborough Village West, Cliffcrest",3,3,3,3,3,3
"Dorset Park, Wexford Heights, Scarborough Town Centre",8,8,8,8,8,8
"East Birchmount Park, Ionview, Kennedy Park",6,6,6,6,6,6
"L'Amoreaux West, Steeles West",14,14,14,14,14,14


In [12]:
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")
scarborough_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]
scarborough_grouped = scarborough_onehot.groupby('Neighborhood').mean().reset_index()
scarborough_grouped

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,...,Rental Car Location,Sandwich Place,Sculpture Garden,Shopping Mall,Skating Rink,Soccer Field,Thai Restaurant,Thrift / Vintage Store,Train Station,Vietnamese Restaurant
0,Agincourt,0.000000,0.000,0.000000,0.000000,0.000,0.000000,0.250000,0.000000,0.000000,...,0.000000,0.250000,0.000000,0.000000,0.250,0.000000,0.000000,0.000,0.000000,0.000
1,"Agincourt North, Steeles East, Milliken, L'Amo...",0.000000,0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.333333,0.000000,0.000,0.000000,0.000000,0.000,0.000000,0.000
2,"Birch Cliff, Cliffside West",0.000000,0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.250,0.000000,0.000000,0.000,0.000000,0.000
3,Cedarbrae,0.000000,0.125,0.000000,0.125000,0.125,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.125,0.000000,0.125000,0.000,0.000000,0.000
4,"Clarks Corners, Sullivan, Tam O'Shanter",0.000000,0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,...,0.111111,0.000000,0.000000,0.000000,0.000,0.000000,0.111111,0.000,0.000000,0.000
5,"Cliffside, Scarborough Village West, Cliffcrest",0.333333,0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000,0.000000,0.000
6,"Dorset Park, Wexford Heights, Scarborough Town...",0.000000,0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.125,0.000000,0.125
7,"East Birchmount Park, Ionview, Kennedy Park",0.000000,0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.166667,...,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000,0.166667,0.000
8,"L'Amoreaux West, Steeles West",0.000000,0.000,0.000000,0.000000,0.000,0.000000,0.071429,0.000000,0.000000,...,0.000000,0.071429,0.000000,0.000000,0.000,0.000000,0.000000,0.000,0.000000,0.000
9,"Maryvale, Wexford",0.000000,0.000,0.142857,0.142857,0.000,0.000000,0.142857,0.000000,0.000000,...,0.000000,0.142857,0.000000,0.142857,0.000,0.000000,0.000000,0.000,0.000000,0.000


In [13]:
import numpy as np
num_top_venues = 17

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

indicators = ['st', 'nd', 'rd']
# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue
0,Agincourt,Skating Rink,Lounge,Sandwich Place,Breakfast Spot,Vietnamese Restaurant,Convenience Store,Hakka Restaurant,Grocery Store,Golf Course,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Coffee Shop
1,"Agincourt North, Steeles East, Milliken, L'Amo...",Sculpture Garden,Playground,Park,Vietnamese Restaurant,College Stadium,Hakka Restaurant,Grocery Store,Golf Course,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,Chinese Restaurant
2,"Birch Cliff, Cliffside West",Skating Rink,General Entertainment,Café,College Stadium,Vietnamese Restaurant,Convenience Store,History Museum,Hakka Restaurant,Grocery Store,Golf Course,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Coffee Shop
3,Cedarbrae,Athletics & Sports,Thai Restaurant,Skating Rink,Bakery,Bank,Hakka Restaurant,Fried Chicken Joint,Caribbean Restaurant,Vietnamese Restaurant,Furniture / Home Store,Electronics Store,Fast Food Restaurant,Grocery Store,General Entertainment,Golf Course,Department Store,History Museum
4,"Clarks Corners, Sullivan, Tam O'Shanter",Pizza Place,Noodle House,Italian Restaurant,Pharmacy,Fried Chicken Joint,Rental Car Location,Thai Restaurant,Chinese Restaurant,General Entertainment,Furniture / Home Store,Golf Course,College Stadium,Fast Food Restaurant,Grocery Store,Electronics Store,Discount Store,Department Store
5,"Cliffside, Scarborough Village West, Cliffcrest",American Restaurant,Motel,Movie Theater,Indian Restaurant,History Museum,Hakka Restaurant,Grocery Store,Golf Course,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium
6,"Dorset Park, Wexford Heights, Scarborough Town...",Indian Restaurant,Pet Store,Latin American Restaurant,Furniture / Home Store,Chinese Restaurant,Vietnamese Restaurant,Thrift / Vintage Store,Bank,Discount Store,History Museum,Hakka Restaurant,Grocery Store,Golf Course,General Entertainment,Athletics & Sports,Fried Chicken Joint,Fast Food Restaurant
7,"East Birchmount Park, Ionview, Kennedy Park",Department Store,Bus Station,Convenience Store,Train Station,Coffee Shop,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Vietnamese Restaurant,General Entertainment,Golf Course,Grocery Store,Hakka Restaurant,History Museum,Ice Cream Shop,Furniture / Home Store
8,"L'Amoreaux West, Steeles West",Chinese Restaurant,Coffee Shop,Fast Food Restaurant,Sandwich Place,Electronics Store,Pharmacy,Pizza Place,Nail Salon,Breakfast Spot,Japanese Restaurant,Grocery Store,Convenience Store,Furniture / Home Store,General Entertainment,Golf Course,Discount Store,Department Store
9,"Maryvale, Wexford",Paper / Office Supplies Store,Shopping Mall,Middle Eastern Restaurant,Sandwich Place,Breakfast Spot,Bakery,Auto Garage,Grocery Store,Golf Course,Convenience Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Hakka Restaurant


#### Cluster the neighborhood Scarborough

In [14]:
from sklearn.cluster import KMeans


# Number of clusters 4
# Using k-means to cluster the neighborhood into 4 clusters.
kclusters = 4

scarborough_grouped_clustering = scarborough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 1, 0], dtype=int32)

In [15]:
scarborough_merged = Scarborough_data[0:16]

# add clustering labels
scarborough_merged['Cluster Labels'] = kmeans.labels_

# merge to add latitude/longitude for each neighborhood
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')


scarborough_merged

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,...,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,1,Fast Food Restaurant,Vietnamese Restaurant,Convenience Store,Ice Cream Shop,...,Golf Course,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Electronics Store,Discount Store,Department Store,College Stadium,Train Station,Coffee Shop
1,M1C,Scarborough,"Port Union, Highland Creek, Rouge Hill",43.784535,-79.160497,1,History Museum,Bar,Golf Course,Vietnamese Restaurant,...,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,College Stadium,Train Station
2,M1E,Scarborough,"West Hill, Guildwood, Morningside",43.763573,-79.188711,1,Rental Car Location,Electronics Store,Mexican Restaurant,Pizza Place,...,Discount Store,Fast Food Restaurant,Fried Chicken Joint,Convenience Store,Furniture / Home Store,General Entertainment,Golf Course,Grocery Store,Hakka Restaurant,Department Store
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1,Coffee Shop,Korean Restaurant,Pharmacy,Vietnamese Restaurant,...,Grocery Store,Golf Course,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,College Stadium
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1,Athletics & Sports,Thai Restaurant,Skating Rink,Bakery,...,Caribbean Restaurant,Vietnamese Restaurant,Furniture / Home Store,Electronics Store,Fast Food Restaurant,Grocery Store,General Entertainment,Golf Course,Department Store,History Museum
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,1,Playground,Vietnamese Restaurant,College Stadium,History Museum,...,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,Coffee Shop,Indian Restaurant
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,1,Department Store,Bus Station,Convenience Store,Train Station,...,Fast Food Restaurant,Fried Chicken Joint,Vietnamese Restaurant,General Entertainment,Golf Course,Grocery Store,Hakka Restaurant,History Museum,Ice Cream Shop,Furniture / Home Store
7,M1L,Scarborough,"Oakridge, Golden Mile, Clairlea",43.711112,-79.284577,1,Bakery,Bus Line,Intersection,Ice Cream Shop,...,Fried Chicken Joint,Electronics Store,Fast Food Restaurant,Furniture / Home Store,Department Store,General Entertainment,Golf Course,Grocery Store,Hakka Restaurant,Discount Store
8,M1M,Scarborough,"Cliffside, Scarborough Village West, Cliffcrest",43.716316,-79.239476,1,American Restaurant,Motel,Movie Theater,Indian Restaurant,...,Golf Course,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,1,Skating Rink,General Entertainment,Café,College Stadium,...,Hakka Restaurant,Grocery Store,Golf Course,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Coffee Shop


In [16]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[43.773077, -79.257774], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
colors_array = cm.rainbow(np.linspace(0, 1, kclusters))
rainbow = [colors.rgb2hex(i) for i in colors_array]
print(rainbow)
# add markers to the map
markers_colors = []
for lat, lon, nei , cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighborhood'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(nei) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

['#8000ff', '#2adddd', '#d4dd80', '#ff0000']


## Examine Clusters

##### Cluster 0

In [17]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 0,scarborough_merged.columns[[2] + list(range(5, scarborough_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue
15,"L'Amoreaux West, Steeles West",0,Chinese Restaurant,Coffee Shop,Fast Food Restaurant,Sandwich Place,Electronics Store,Pharmacy,Pizza Place,Nail Salon,Breakfast Spot,Japanese Restaurant,Grocery Store,Convenience Store,Furniture / Home Store,General Entertainment,Golf Course,Discount Store,Department Store


##### Cluster 1

In [18]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 1,scarborough_merged.columns[[2] + list(range(5, scarborough_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue
0,"Rouge, Malvern",1,Fast Food Restaurant,Vietnamese Restaurant,Convenience Store,Ice Cream Shop,History Museum,Hakka Restaurant,Grocery Store,Golf Course,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Electronics Store,Discount Store,Department Store,College Stadium,Train Station,Coffee Shop
1,"Port Union, Highland Creek, Rouge Hill",1,History Museum,Bar,Golf Course,Vietnamese Restaurant,Convenience Store,Ice Cream Shop,Hakka Restaurant,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,College Stadium,Train Station
2,"West Hill, Guildwood, Morningside",1,Rental Car Location,Electronics Store,Mexican Restaurant,Pizza Place,Breakfast Spot,Medical Center,Vietnamese Restaurant,Discount Store,Fast Food Restaurant,Fried Chicken Joint,Convenience Store,Furniture / Home Store,General Entertainment,Golf Course,Grocery Store,Hakka Restaurant,Department Store
3,Woburn,1,Coffee Shop,Korean Restaurant,Pharmacy,Vietnamese Restaurant,Convenience Store,History Museum,Hakka Restaurant,Grocery Store,Golf Course,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,College Stadium
4,Cedarbrae,1,Athletics & Sports,Thai Restaurant,Skating Rink,Bakery,Bank,Hakka Restaurant,Fried Chicken Joint,Caribbean Restaurant,Vietnamese Restaurant,Furniture / Home Store,Electronics Store,Fast Food Restaurant,Grocery Store,General Entertainment,Golf Course,Department Store,History Museum
5,Scarborough Village,1,Playground,Vietnamese Restaurant,College Stadium,History Museum,Hakka Restaurant,Grocery Store,Golf Course,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,Coffee Shop,Indian Restaurant
6,"East Birchmount Park, Ionview, Kennedy Park",1,Department Store,Bus Station,Convenience Store,Train Station,Coffee Shop,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Vietnamese Restaurant,General Entertainment,Golf Course,Grocery Store,Hakka Restaurant,History Museum,Ice Cream Shop,Furniture / Home Store
7,"Oakridge, Golden Mile, Clairlea",1,Bakery,Bus Line,Intersection,Ice Cream Shop,Park,Soccer Field,Metro Station,Fried Chicken Joint,Electronics Store,Fast Food Restaurant,Furniture / Home Store,Department Store,General Entertainment,Golf Course,Grocery Store,Hakka Restaurant,Discount Store
8,"Cliffside, Scarborough Village West, Cliffcrest",1,American Restaurant,Motel,Movie Theater,Indian Restaurant,History Museum,Hakka Restaurant,Grocery Store,Golf Course,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium
9,"Birch Cliff, Cliffside West",1,Skating Rink,General Entertainment,Café,College Stadium,Vietnamese Restaurant,Convenience Store,History Museum,Hakka Restaurant,Grocery Store,Golf Course,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Coffee Shop


##### Cluster 2

In [19]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 2,scarborough_merged.columns[[2] + list(range(5, scarborough_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue
12,Agincourt,2,Skating Rink,Lounge,Sandwich Place,Breakfast Spot,Vietnamese Restaurant,Convenience Store,Hakka Restaurant,Grocery Store,Golf Course,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Coffee Shop


##### Cluster 3

In [20]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 3,scarborough_merged.columns[[2] + list(range(5, scarborough_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue
13,"Clarks Corners, Sullivan, Tam O'Shanter",3,Pizza Place,Noodle House,Italian Restaurant,Pharmacy,Fried Chicken Joint,Rental Car Location,Thai Restaurant,Chinese Restaurant,General Entertainment,Furniture / Home Store,Golf Course,College Stadium,Fast Food Restaurant,Grocery Store,Electronics Store,Discount Store,Department Store


#### Thats is all